In [17]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from shap import sample as shap_sample

def generate_shap_explanation(exog_train, exog_test, model_results, exog_vars, product, geography, sample_size=100, save_dir="shap_outputs"):
    """
    Generate and visualize SHAP values for a SARIMAX model.

    Parameters:
    - exog_train: DataFrame of training exogenous variables
    - exog_test: DataFrame of testing exogenous variables
    - model_results: Trained SARIMAX model result object
    - exog_vars: List of exogenous variable names
    - product: Product identifier (for title/file name)
    - geography: Geography identifier (for title/file name)
    - sample_size: Sample size for SHAP explanation (default: 100)
    - save_dir: Directory to save the SHAP plot (default: 'shap_outputs')

    Returns:
    - DataFrame of SHAP feature importances
    """
    
    # Ensure sample size is feasible
    sample_size = min(sample_size, len(exog_test))
    if sample_size < 30:
        print(f"Skipping SHAP for ({product}, {geography}): Not enough test data.")
        return None

    # Sample background data
    background = shap_sample(exog_train, sample_size)

    # Define model prediction wrapper
    def model_predict(X):
        X_df = pd.DataFrame(X, columns=exog_vars)
        preds = model_results.get_prediction(start=0, end=len(X_df)-1, exog=X_df)
        return preds.predicted_mean.values

    # Initialize SHAP explainer
    explainer = shap.KernelExplainer(model_predict, background)

    # Compute SHAP values
    shap_values = explainer.shap_values(exog_test[:sample_size], nsamples=sample_size)

    # Calculate mean absolute SHAP values
    importances = np.abs(shap_values).mean(axis=0)
    importance_df = pd.DataFrame({
        'Feature': exog_vars,
        'Mean_Abs_SHAP': importances
    }).sort_values(by='Mean_Abs_SHAP', ascending=True)

    # Create output directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"shap_bar_summary_{product}_{geography}.png")

    # Plot
    plt.figure(figsize=(6, 3))
    plt.barh(importance_df['Feature'], importance_df['Mean_Abs_SHAP'])
    plt.xlabel("Mean |SHAP value|")
    plt.title(f"SHAP Bar Summary: {product} - {geography}")
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

    return importance_df

In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import seaborn as sns
pd.set_option('display.max_columns', None)

# Load Data
df = pd.read_excel("stockist_data_with_date3 3.xlsx")

In [19]:
df.shape

(30000, 31)

In [20]:
df.head()

,Partner_id,AIS(Air Insulated Switchgear),RMU(Ring Main Unit),PSS(Compact Sub-Stations),VCU(Vacuum Contactor Units),E-House,VCB(Vacuum Circuit Breaker),ACB(Air Circuit Breaker),MCCB(Moduled Case Circuit Breaker),SDF(Switch Disconnectors),BBT(Busbar Trunking),Modular Switches,Starter,Controller,Solar Solutions,Pump Starter and Controller,Geography,Stockist_Type,Scheme_Type,Sales_Value_Last_Period,Sales_Quantity_Last_Period,MRP,Discount_Applied,Bulk_Purchase_Tendency,Competitor_Price,Competitor_Discount,Date,Month,Year,Sales_Value_Last_Period_3M_Ago,Discount_Applied_3M_Ago
0,281_74,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,West,Retailer,Bulk Purchase,22014,2,4679,5,0,914.92,7,2024-12-10,12,2024,18426.25864,0
1,549_85,0,1,0,1,1,1,0,1,0,1,1,1,0,1,1,North,Wholesaler,Bulk Purchase,28863,2,3335,7,0,915.17,9,2025-01-04,1,2025,25648.28893,6
2,742_66,1,0,0,0,0,1,0,1,1,1,1,1,1,0,1,Central,Wholesaler,Discount,13729,2,1065,2,1,917.96,0,2022-10-23,10,2022,16393.81271,0
3,199_92,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,Central,Retailer,Bulk Purchase,22038,2,1244,5,1,918.68,1,2022-09-17,9,2022,25221.93771,0
4,412_67,1,1,0,1,0,1,1,1,0,0,1,0,0,1,0,Central,Wholesaler,Loyalty,26075,2,4604,2,0,919.33,3,2023-03-23,3,2023,27523.66971,0


In [21]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split

# Ensure Date column is properly converted to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # 'coerce' will convert invalid dates to NaT

# Drop rows with invalid dates if any
df = df.dropna(subset=['Date'])

# Sort by date
df = df.sort_values(by='Date')

# Extract relevant product categories (using your original column names)
expected_product_columns = [
    'AIS(Air Insulated Switchgear)', 'RMU(Ring Main Unit)', 
    'PSS(Compact Sub-Stations)', 'VCU(Vacuum Contactor Units)', 
    'E-House', 'VCB(Vacuum Circuit Breaker)', 
    'ACB(Air Circuit Breaker)', 'MCCB(Moduled Case Circuit Breaker)', 
    'SDF(Switch Disconnectors)', 'BBT(Busbar Trunking)', 
    'Modular Switches', 'Starter', 'Controller', 
    'Solar Solutions', 'Pump Starter and Controller'
]

# Find which of these columns actually exist in the dataframe
product_columns = [col for col in expected_product_columns if col in df.columns]

# Create Product_id based on which product has value 1
df['Product_id'] = df[product_columns].idxmax(axis=1)

# Selecting necessary columns
df = df[['Partner_id', 'Product_id', 'Date', 'MRP', 'Sales_Quantity_Last_Period', 
         'Discount_Applied', 'Geography', 'Competitor_Price']].dropna()

# Rename columns
df.rename(columns={
    'MRP': 'Price', 
    'Sales_Quantity_Last_Period': 'Demand', 
    'Discount_Applied': 'Discount'
}, inplace=True)

# Filter valid data
df = df[(df['Price'] > 0) & (df['Demand'] > 0)]

In [22]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split


def prepare_data(df):
    """Prepare and clean the input dataframe with proper date handling"""
    # Convert and validate dates
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')
    
    # Create proper monthly index (ensuring correct period conversion)
    df['YearMonth'] = pd.to_datetime(df['Date'].dt.to_period('M').astype(str))

    # Set as datetime index
    df.set_index('YearMonth', inplace=True)


    
    # Aggregate to monthly level
    monthly_df = df.groupby(['Product_id', 'Geography', 'YearMonth']).agg({
        'Demand': 'sum',
        'Price': 'mean',
        'Competitor_Price': 'mean',
        'Discount': 'mean'
    }).reset_index()
    
    # Log transformations with proper handling
    for col in ['Demand', 'Price', 'Competitor_Price']:
        monthly_df[f'Log_{col}'] = np.log1p(monthly_df[col])  # Using log1p to handle zeros
    
    return monthly_df.dropna()

In [23]:
df

,Partner_id,Product_id,Date,Price,Demand,Discount,Geography,Competitor_Price
22601,22602_67,RMU(Ring Main Unit),2020-03-05,4227,76,3,East,4195.81
2227,02228_86,VCU(Vacuum Contactor Units),2020-03-05,3432,8,4,North,1360.15
29628,29629_69,AIS(Air Insulated Switchgear),2020-03-05,4793,99,5,South,5597.62
5021,05022_87,AIS(Air Insulated Switchgear),2020-03-05,2588,23,25,South,1754.46
23931,23932_27,RMU(Ring Main Unit),2020-03-05,1937,80,9,South,4390.01
...,...,...,...,...,...,...,...,...
11350,11351_89,PSS(Compact Sub-Stations),2025-03-04,4773,39,5,South,2624.43
4532,04533_23,AIS(Air Insulated Switchgear),2025-03-04,4997,23,7,South,1686.44
3496,03497_50,VCU(Vacuum Contactor Units),2025-03-04,3023,22,5,East,1538.38
8523,08524_58,AIS(Air Insulated Switchgear),2025-03-04,3934,29,6,East,2232.02


In [24]:
# Prepare the data
monthly_df = prepare_data(df)

In [25]:
monthly_df

,Product_id,Geography,YearMonth,Demand,Price,Competitor_Price,Discount,Log_Demand,Log_Price,Log_Competitor_Price
0,ACB(Air Circuit Breaker),Central,2020-03-01,35,2394.000000,2470.590000,9.000000,3.583519,7.781139,7.812617
1,ACB(Air Circuit Breaker),Central,2020-04-01,53,4197.000000,3252.380000,25.000000,3.988984,8.342364,8.087450
2,ACB(Air Circuit Breaker),Central,2020-05-01,67,1836.000000,3824.290000,8.000000,4.219508,7.515889,8.249390
3,ACB(Air Circuit Breaker),Central,2020-06-01,76,1125.000000,4190.790000,7.000000,4.343805,7.026427,8.340883
4,ACB(Air Circuit Breaker),Central,2020-07-01,88,2453.500000,2858.535000,6.000000,4.488636,7.805678,7.958414
...,...,...,...,...,...,...,...,...,...,...
2099,VCU(Vacuum Contactor Units),West,2024-10-01,429,3509.500000,3257.976250,12.625000,6.063785,8.163514,8.089168
2100,VCU(Vacuum Contactor Units),West,2024-11-01,365,3901.428571,3187.494286,11.428571,5.902633,8.269354,8.067304
2101,VCU(Vacuum Contactor Units),West,2024-12-01,276,2903.500000,2419.586250,4.250000,5.624018,7.974017,7.791765
2102,VCU(Vacuum Contactor Units),West,2025-01-01,310,2945.800000,3605.706000,4.400000,5.739793,7.988475,8.190550


In [26]:
monthly_df

,Product_id,Geography,YearMonth,Demand,Price,Competitor_Price,Discount,Log_Demand,Log_Price,Log_Competitor_Price
0,ACB(Air Circuit Breaker),Central,2020-03-01,35,2394.000000,2470.590000,9.000000,3.583519,7.781139,7.812617
1,ACB(Air Circuit Breaker),Central,2020-04-01,53,4197.000000,3252.380000,25.000000,3.988984,8.342364,8.087450
2,ACB(Air Circuit Breaker),Central,2020-05-01,67,1836.000000,3824.290000,8.000000,4.219508,7.515889,8.249390
3,ACB(Air Circuit Breaker),Central,2020-06-01,76,1125.000000,4190.790000,7.000000,4.343805,7.026427,8.340883
4,ACB(Air Circuit Breaker),Central,2020-07-01,88,2453.500000,2858.535000,6.000000,4.488636,7.805678,7.958414
...,...,...,...,...,...,...,...,...,...,...
2099,VCU(Vacuum Contactor Units),West,2024-10-01,429,3509.500000,3257.976250,12.625000,6.063785,8.163514,8.089168
2100,VCU(Vacuum Contactor Units),West,2024-11-01,365,3901.428571,3187.494286,11.428571,5.902633,8.269354,8.067304
2101,VCU(Vacuum Contactor Units),West,2024-12-01,276,2903.500000,2419.586250,4.250000,5.624018,7.974017,7.791765
2102,VCU(Vacuum Contactor Units),West,2025-01-01,310,2945.800000,3605.706000,4.400000,5.739793,7.988475,8.190550


In [27]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_test_split_monthly(df, test_size=0.2):
    """
    Time-based train-test split that maintains temporal order
    """
    # Convert to datetime if not already
    df['YearMonth'] = pd.to_datetime(df['YearMonth'])
    
    # Get all unique months and sort them
    unique_months = pd.Series(df['YearMonth'].unique()).sort_values().values
    
    # Calculate split point
    split_idx = int(len(unique_months) * (1 - test_size))
    train_months = unique_months[:split_idx]
    test_months = unique_months[split_idx:]
    
    # Split the data
    train_df = df[df['YearMonth'].isin(train_months)]
    test_df = df[df['YearMonth'].isin(test_months)]
    
    return train_df, test_df



# from statsmodels.tsa.statespace.sarimax import SARIMAX

# def train_sarimax_model(train_group):
#     """Train SARIMAX model on training data only"""
#     try:
#         # Prevent modifying original data
#         train_group = train_group.copy().sort_values('YearMonth')

#         print("Min values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].min())
#         print("Max values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].max())
#         print("Check for NaN:\n", train_group.isna().sum())
        

#         # Ensure proper index format
#         train_group['YearMonth'] = pd.to_datetime(train_group['YearMonth'])
#         train_group.set_index('YearMonth', inplace=True)
#         # train_group = train_group.resample('MS').asfreq()
        
#         train_group.index = pd.date_range(start=train_group.index.min(), 
#                                   periods=len(train_group), 
#                                   freq='MS')

#         # Extract target variable
#         y_train = train_group['Log_Demand'].dropna()
#         print("Unique time periods:", len(y_train))

#         # Extract exogenous variables (ensure no missing values)
#         exog_cols = ['Log_Price', 'Log_Competitor_Price']
#         exog_train = train_group[exog_cols].dropna()

#         # Skip if insufficient training data
#         if len(y_train) < 24:
#             print("Skipping training: Insufficient data.")
#             return None

#         # Train SARIMAX model
#         model = SARIMAX(
#             y_train,
#             exog=exog_train if not exog_train.empty else None,  # Handle case where exog might be empty
#             order=(1, 1, 1),
#             seasonal_order=(0, 1, 1, 6),
#             enforce_stationarity=False,
#             enforce_invertibility=False 
#         )
#         # fitted_model = model.fit(disp=False)
#         fitted_model = model.fit(method='powell', disp=False)


#         return fitted_model

#     except Exception as e:
#         print(f"Training failed: {str(e)}")
#         return None


def train_sarimax_model(train_group):
    """Train SARIMAX model on training data only"""
    try:
        # Prevent modifying original data
        train_group = train_group.copy().sort_values('YearMonth')

        # # Debugging checks
        print("Min values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].min())
        print("Max values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].max())
        print("Check for NaN:\n", train_group.isna().sum())

        # Ensure proper index format
        train_group['YearMonth'] = pd.to_datetime(train_group['YearMonth'])
        train_group.set_index('YearMonth', inplace=True)

        # Ensure monthly frequency without modifying actual timestamps
        train_group = train_group.asfreq('MS')

        # Extract target variable
        y_train = train_group['Log_Demand']
        # y_train = train_group['Log_Demand'].dropna()
        # print("Unique time periods:", len(y_train))

        # Extract exogenous variables, filling missing values
        exog_cols = ['Log_Price', 'Log_Competitor_Price']
        exog_train = train_group[exog_cols].ffill()  # Forward fill exog vars


        # Skip if insufficient training data
        if len(y_train) < 24:
            print("Skipping training: Insufficient data.")
            return None

        # Train SARIMAX model
        model = SARIMAX(
            y_train,
            exog=exog_train if not exog_train.empty else None,  
            order=(1, 1, 1),
            seasonal_order=(0, 1, 1, 12),  # 12-month seasonality (was 6)
            enforce_stationarity=True,  # Force stability
            enforce_invertibility=True   # Prevent large errors
        )

        start_params = [
        0.5,    # AR(1)
        0.3,    # MA(1)
        0.2,    # Seasonal MA(1)
        0.1,    # Intercept
        0.05,   # Coef for exog 1
        -0.05,  # Coef for exog 2
        1.0     # sigma² (variance of errors)
        ]
        # Fit model with more stable optimization
        fitted_model = model.fit()
        # fitted_model = model.fit(start_params = start_params, method='lbfgs', maxiter=200, disp=False)

        return fitted_model
    
    except Exception as e:
        print(f"Training failed: {str(e)}")
        return None



def calculate_elasticity(fitted_model):
    """Calculates price elasticity from fitted model"""
    try:
        # Get elasticity (coefficient of Log_Price)
        if hasattr(fitted_model, 'params') and 'Log_Price' in fitted_model.params:
            return fitted_model.params['Log_Price']
        return np.nan
    except Exception as e:
        print(f"Elasticity calculation failed: {str(e)}")
        return np.nan
    

def evaluate_model(model, test_group):
    """Evaluate model performance on test data"""
    try:
        # Clean test data
        # test_group = clean_log_values(test_group.copy())
        test_group = test_group.sort_values('YearMonth')
        
        # Prepare test data
        y_test = test_group.set_index('YearMonth')['Log_Demand']
        exog_test = test_group.set_index('YearMonth')[['Log_Price', 'Log_Competitor_Price']]
        
        # Generate forecasts
        forecast = model.get_forecast(
            steps=len(y_test),
            exog=exog_test
        )
        
        # Calculate metrics (converting back from log scale)
        pred = np.exp(forecast.predicted_mean).values  # Convert to numpy array
        actual = np.exp(y_test).values  # Convert to numpy array
        
        mae = mean_absolute_error(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore", RuntimeWarning)
        mape = np.mean(np.abs((actual - pred) / actual)) * 100
        
        return {
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'actual': actual,
            'predicted': pred
        }
    
    except Exception as e:
        print(f"Evaluation failed: {str(e)}")
        return None

    
def generate_forecasts(model, train_group, steps=12):
    """Generate future forecasts with proper unit handling"""
    try:
        # Get last available data point
        last_data = train_group.sort_values('YearMonth').iloc[-1]
        
        # Create future exogenous variables
        future_exog = pd.DataFrame(
            [last_data[['Log_Price', 'Log_Competitor_Price']].values] * steps,
            columns=['Log_Price', 'Log_Competitor_Price'],
            index=pd.date_range(
                start=train_group['YearMonth'].max() + pd.offsets.MonthBegin(1),
                periods=steps,
                freq='MS'
            )
        )
        
        # Generate forecasts
        forecast = model.get_forecast(
            steps=steps,
            exog=future_exog
        ).predicted_mean
        
        # Return in original scale (only convert if model used log)
        if 'Log_Demand' in model.model.endog_names:
            return np.exp(forecast)
        return forecast
    
    except Exception as e:
        print(f"Forecast generation failed: {str(e)}")
        return None

def run_forecast_pipeline(monthly_df):
    """Complete forecasting pipeline"""
    # 1. Split data (time-based)
    train_df, test_df = train_test_split_monthly(monthly_df, test_size=0.2)
    
    detailed_results = []
    performance_metrics = []
    
    for (product, region), group in train_df.groupby(['Product_id', 'Geography']):
        # Get corresponding test data
        test_group = test_df[
            (test_df['Product_id'] == product) & 
            (test_df['Geography'] == region)
        ]
        
        # Skip if no test data
        if len(test_group) == 0:
            continue
        
        # Train model
        model = train_sarimax_model(group)
        if model is None:
            continue

        elasticity = calculate_elasticity(model)
        
        # Evaluate model
        eval_results = evaluate_model(model, test_group)
        if eval_results is None:
            continue
        
        # Generate forecasts
        forecasts = generate_forecasts(model, group)
        
        # Store results
        detailed_results.append({
            'product': product,
            'region': region,
            'model': model,
            'actual': eval_results['actual'],
            'predicted': eval_results['predicted'],
            'forecast': forecasts, 
            'elasticity': elasticity
        })
        
        performance_metrics.append({
            'product': product,
            'region': region,
            'mae': eval_results['mae'],
            'rmse': eval_results['rmse'],
            'mape': eval_results['mape']
        })
    
    return pd.DataFrame(performance_metrics), detailed_results

In [28]:
# Prepare your monthly data (ensure it has Log_Demand, Log_Price, Log_Competitor_Price columns)
monthly_df = prepare_data(df)

# Run the full pipeline
metrics_df, detailed_results = run_forecast_pipeline(monthly_df)

# Analyze results
print(metrics_df)  # Performance metrics for all product-region combinations

Min values:
 Log_Demand              1.609438
Log_Price               7.026427
Log_Competitor_Price    7.106147
dtype: float64
Max values:
 Log_Demand              5.283204
Log_Price               8.497603
Log_Competitor_Price    8.544283
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.09433D+08    |proj g|=  9.67990D+12


/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


  ys=-8.788E-10  -gs= 4.975E-06 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    5    f=  9.33904D-01    |proj g|=  1.13386D-01

At iterate   10    f=  8.49539D-01    |proj g|=  5.98832D-02

At iterate   15    f=  8.46435D-01    |proj g|=  2.34323D-03

At iterate   20    f=  8.46106D-01    |proj g|=  1.65803D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     22     82      2     1     0   6.723D-05   8.461D-01
  F =  0.84610610526002439     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Min values:
 Log_Demand              1.386294
Log_Price               6.964136
Log_Competitor_Price    7.055063
dtype: float64
Max values:
 Log_Demand              5.093750
Lo

...


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     38     64      1     0     0   2.121D-04   5.979D-01
  F =  0.59793592456843281     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Min values:
 Log_Demand              4.465908
Log_Price               7.180451
Log_Competitor_Price    7.741761
dtype: float64
Max values:
 Log_Demand              6.587550
Log_Price               8.235957
Log_Competitor_Price    8.375777
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price       

/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.58290D-01    |proj g|=  2.32781D-01

At iterate    5    f=  5.11252D-01    |proj g|=  5.19136D-02

At iterate   10    f=  5.01335D-01    |proj g|=  1.55983D-02

At iterate   15    f=  4.98460D-01    |proj g|=  2.40769D-03

At iterate   20    f=  4.98187D-01    |proj g|=  7.68905D-03

At iterate   25    f=  4.98147D-01    |proj g|=  1.40330D-03

At iterate   30    f=  4.98143D-01    |proj g|=  1.89353D-03

At iterate   35    f=  4.98143D-01    |proj g|=  2.59499D-04

At iterate   40    f=  4.98143D-01    |proj g|=  5.84392D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.



At iterate    5    f=  6.83857D-01    |proj g|=  8.13030D-02

At iterate   10    f=  6.76352D-01    |proj g|=  5.57629D-03

At iterate   15    f=  6.75966D-01    |proj g|=  3.59995D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     19     23      1     0     0   6.393D-06   6.760D-01
  F =  0.67596487304046615     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                               product   region           mae          rmse  \
0             ACB(Air Circuit Breaker)  Central     58.823194     70.779628   
1             ACB(Air Circuit Breaker)     East     15.621431     25.845803   


In [29]:
print(detailed_results)

[{'product': 'ACB(Air Circuit Breaker)', 'region': 'Central', 'model': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x77a088e6fc10>, 'actual': array([124.,  90.,  78.,  41.]), 'predicted': array([ 9.70055428, 94.71693731, 10.05592645, 89.33232141]), 'forecast': 2024-02-01     6.618878
2024-03-01    72.148625
2024-04-01     5.437716
2024-05-01    79.903726
2024-06-01     5.711668
2024-07-01    19.769626
2024-08-01    17.629437
2024-09-01    87.957721
2024-10-01     4.119612
2024-11-01    64.336859
2024-12-01    28.082127
2025-01-01    18.276066
Freq: MS, Name: predicted_mean, dtype: float64, 'elasticity': -0.2224760436894028}, {'product': 'ACB(Air Circuit Breaker)', 'region': 'East', 'model': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x77a08890fc90>, 'actual': array([ 31.,  27.,  63., 156.,  70.,  38.,  10.]), 'predicted': array([25.68628084, 35.85951894, 64.01643564, 92.6793471 , 47.77412814,
       38.58633007, 18.02749166]), 'forecast'

In [30]:
def save_results_to_csv(metrics_df, detailed_results, base_filename="forecast_results"):
    """
    Save forecast results to CSV files
    - Ensures proper date formatting
    - Preserves original units (no double conversion)
    - Creates clean, analysis-ready outputs
    """
    
    # 1. Save performance metrics
    metrics_df.to_csv(f"{base_filename}_metrics.csv", index=False)
    
    # 2. Save forecasts (12-month predictions)
    forecast_data = []
    elasticity_data = []
    for result in detailed_results:
        if result['forecast'] is None:
            continue
            
        # Create future dates starting next month
        last_date = result['actual'].index[-1] if isinstance(result['actual'], pd.Series) else pd.Timestamp.now()
        dates = pd.date_range(
            start=last_date + pd.offsets.MonthBegin(1),
            periods=len(result['forecast']),
            freq='MS'
        )
        
        forecast_data.append(pd.DataFrame({
            'product': result['product'],
            'region': result['region'],
            'date': dates.strftime('%Y-%m-%d'),
            'forecast': result['forecast']  # Already in original units
        }))
        # Elasticity DataFrame (simple product-region mapping)
        if 'elasticity' in result:
            elasticity_data.append({
                'product': result['product'],
                'region': result['region'], 
                'price_elasticity': result['elasticity']
            })
    
    if forecast_data:
        pd.concat(forecast_data).to_csv(f"{base_filename}_forecasts.csv", index=False)
    
    # 3. Save actual vs predicted comparisons
    comparison_data = []
    for result in detailed_results:
        if result.get('actual') is None or result.get('predicted') is None:
            continue
            
        # Get existing dates or create default range
        if isinstance(result['actual'], pd.Series):
            dates = result['actual'].index
        else:
            dates = pd.date_range(
                end=pd.Timestamp.now(),
                periods=len(result['actual']),
                freq='MS'
            )
        
        comparison_data.append(pd.DataFrame({
            'product': result['product'],
            'region': result['region'],
            'elasticity': result['elasticity'],
            'date': dates.strftime('%Y-%m-%d'),
            'actual': result['actual'],  # Already in original units
            'predicted': result['predicted']  # Already in original units
        }))
    
    if comparison_data:
        pd.concat(comparison_data).to_csv(f"{base_filename}_comparisons.csv", index=False)

    if elasticity_data:
        pd.DataFrame(elasticity_data).to_csv(f"{base_filename}_elasticity.csv", index=False)
    
    print(f"Successfully saved results to:")
    print(f"- Metrics: {base_filename}_metrics.csv")
    print(f"- Forecasts: {base_filename}_forecasts.csv")
    print(f"- Comparisons: {base_filename}_comparisons.csv")
    print(f"- Elasticity: {base_filename}_elasticity.csv")

In [31]:
# With custom filename:
save_results_to_csv(metrics_df, detailed_results, "my_product_forecasts3")

Successfully saved results to:
- Metrics: my_product_forecasts3_metrics.csv
- Forecasts: my_product_forecasts3_forecasts.csv
- Comparisons: my_product_forecasts3_comparisons.csv
- Elasticity: my_product_forecasts3_elasticity.csv


In [32]:
def create_price_discount_simulation(product_region_data, elasticity_value, product_name, region_name):
    """
    Create simulation table with:
    - 5 equidistant price buckets between min/max observed prices
    - Average demand calculated for each price bucket
    - Demand projections at 0-50% discounts (5% intervals)
    
    Args:
        product_region_data: DataFrame with historical transactions
        elasticity_value: Pre-calculated elasticity coefficient
        product_name: Product to analyze
        region_name: Region to analyze
    
    Returns:
        DataFrame with simulation scenarios
    """
    
    # Filter for product-region
    data = product_region_data[
        (product_region_data['Product_id'] == product_name) & 
        (product_region_data['Geography'] == region_name)
    ].copy()
    
    if data.empty:
        print(f"No data for {product_name} in {region_name}")
        return None
    
    # Create 4 price buckets
    min_price = data['Price'].min()
    max_price = data['Price'].max()
    price_buckets = np.linspace(min_price, max_price, 5)
    
    # Calculate average demand per price bucket
    data['price_bucket'] = pd.cut(data['Price'], bins=price_buckets, include_lowest=True)
    bucket_stats = data.groupby('price_bucket').agg(
        avg_demand=('Demand', 'mean'),
        price_midpoint=('Price', lambda x: (x.min() + x.max())/2)
    ).reset_index()
    
    # Generate all discount scenarios (0-50% in 5% steps)
    discount_rates = np.arange(0, 55, 5)
    
    # Build simulation scenarios
    simulation_results = []
    for _, bucket in bucket_stats.iterrows():
        for discount in discount_rates:
            new_demand = calculate_new_demand(
                demand=bucket['avg_demand'],
                elasticity=elasticity_value,
                discount=discount
            )
            
            simulation_results.append({
                'product': product_name,
                'region': region_name,
                'price_bucket': f"{bucket['price_bucket'].left:.2f}-{bucket['price_bucket'].right:.2f}",
                'price_midpoint': round(bucket['price_midpoint'], 2),
                'discount_pct': discount,
                'original_avg_demand': round(bucket['avg_demand'], 2),
                'predicted_demand': round(new_demand, 2),
                'demand_change_pct': round((new_demand - bucket['avg_demand'])/bucket['avg_demand']*100, 1),
                'elasticity_used': round(elasticity_value, 3)
            })
    
    return pd.DataFrame(simulation_results)

def calculate_new_demand(demand, elasticity, discount):
    """Calculate demand response to price changes"""
    discount = np.clip(discount, 0, 50)
    if elasticity is None or np.isnan(elasticity):
        elasticity = -0.2
    return demand * np.exp(elasticity * np.log(1 - discount/100))

In [33]:
# Load your data and elasticity
elasticity_df = pd.read_csv("my_product_forecasts3_elasticity.csv")
product_elasticity = elasticity_df[
    (elasticity_df['product'] == "RMU(Ring Main Unit)") & 
    (elasticity_df['region'] == "West")
]['price_elasticity'].values[0]

# Generate simulation
simulation = create_price_discount_simulation(
    product_region_data=monthly_df,
    elasticity_value=product_elasticity,
    product_name="RMU(Ring Main Unit)",
    region_name="West"
)

# Save results
simulation.to_csv("rmu_west_simulation.csv", index=False)

In [34]:
def calculate_optimal_discounts(simulation_df):
    """
    Calculate profit-optimizing discount at each price point
    Returns DataFrame with optimal discount scenarios
    """
    
    # Calculate revenue for each scenario
    simulation_df['revenue'] = (
        simulation_df['price_midpoint'] * 
        (1 - simulation_df['discount_pct']/100) * 
        simulation_df['predicted_demand']
    )
    
    # Find discount that maximizes revenue for each price bucket
    optimal_discounts = (
        simulation_df.loc[simulation_df.groupby('price_bucket')['revenue'].idxmax()]
        [['product', 'region', 'price_bucket', 'price_midpoint', 
          'discount_pct', 'predicted_demand', 'revenue']]
        .sort_values('price_midpoint')
        .rename(columns={
            'discount_pct': 'optimal_discount_pct',
            'predicted_demand': 'optimal_demand',
            'revenue': 'max_revenue'
        })
    )
    
    return optimal_discounts

In [35]:
## sign of elasticity - 
## 

In [36]:

def save_optimal_discounts(simulation_df, filename="optimal_discounts.csv"):
    """Calculate and save optimal discount scenarios"""
    optimal_df = calculate_optimal_discounts(simulation_df)
    
    # Format output
    optimal_df['price_midpoint'] = optimal_df['price_midpoint'].round(2)
    optimal_df['max_revenue'] = optimal_df['max_revenue'].round(2)
    
    # Save to CSV
    optimal_df.to_csv(filename, index=False)
    print(f"Saved optimal discounts to {filename}")
    return optimal_df

In [37]:
# 2. Calculate and save optimal discounts
optimal_discounts = save_optimal_discounts(
    simulation,
    filename="rmu_west_optimal_discounts.csv"
)

# Display sample results
print(optimal_discounts.head())

Saved optimal discounts to rmu_west_optimal_discounts.csv
                product region     price_bucket  price_midpoint  \
0   RMU(Ring Main Unit)   West  2537.96-2784.72         2659.17   
11  RMU(Ring Main Unit)   West  2784.72-3031.48         2908.06   
22  RMU(Ring Main Unit)   West  3031.48-3278.24         3180.61   
33  RMU(Ring Main Unit)   West  3278.24-3525.00         3404.03   

    optimal_discount_pct  optimal_demand  max_revenue  
0                      0         1168.08   3106123.29  
11                     0         1365.86   3972002.83  
22                     0         1284.83   4086543.15  
33                     0         1434.75   4883932.04  
